# 패키지 임포트

In [1]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
import os
import string
import re
import json
import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from keras.models import load_model
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import sys
sys.path.append('C:/Users/User/Desktop/20Newsgroups')
from data_preprocessing import *

# 데이터 불러오기

In [2]:
df_newsgroup = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/20news_dataset_clear/20newsgroup_preprocessed.csv', sep=';', usecols=['target', 'text_cleaned'])
df_newsgroup.rename(columns={'text_cleaned' : 'text'}, inplace=True)

In [ ]:
textcuboid=np.load('./1-Channel textcuboid_20ng(bert).npy')
textcuboid_test=np.load('./1-Channel textcuboid_test_20ng(bert).npy')

# 데이터 전처리

In [3]:
le = LabelEncoder()
le.fit(df_newsgroup['target'].unique())

LabelEncoder()

In [4]:
df_newsgroup['target'] = le.transform(df_newsgroup['target'])

In [5]:
X = df_newsgroup['text'].astype(str)
y=list(df_newsgroup['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df_newsgroup['target'])

In [6]:
one_hot_train_labels = to_categorical(y_train)
one_hot_test_labels = to_categorical(y_test)

In [ ]:
x_train=textcuboid[1000:]
x_val=textcuboid[:1000]
y_train=one_hot_train_labels[1000:]
y_val=one_hot_train_labels[:1000]

# 모델 빌드

In [7]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model_{epoch}.h5", save_best_only=False, period=5
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
]

In [8]:
input_layer = Input(shape=(155, 768))
conv1 = Conv1D(512, 1, padding='valid', activation='relu')(input_layer)
pooling = GlobalMaxPooling1D()(conv1)

x = Dense(1024, activation='relu')(pooling)
x = Dropout(0.5)(x)
output_layer = Dense(20, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 155, 768)]        0         
                                                                 
 conv1d (Conv1D)             (None, 155, 512)          393728    
                                                                 
 global_max_pooling1d (Globa  (None, 512)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1024)              525312    
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 20)                20500     
                                                             

# 모델 학습

In [15]:
history = model.fit(x_train, y_train,callbacks=callbacks, epochs=50,batch_size=256,validation_data=(x_val, y_val))

Epoch 1/50
55/55 [==============================] - 3s 45ms/step - loss: 4.3070 - accuracy: 0.2587 - val_loss: 1.5028 - val_accuracy: 0.5520 - lr: 0.0010
Epoch 2/50
55/55 [==============================] - 2s 37ms/step - loss: 1.4443 - accuracy: 0.5349 - val_loss: 1.1703 - val_accuracy: 0.6500 - lr: 0.0010
Epoch 3/50
55/55 [==============================] - 2s 35ms/step - loss: 1.0919 - accuracy: 0.6553 - val_loss: 1.0171 - val_accuracy: 0.6940 - lr: 0.0010
Epoch 4/50
55/55 [==============================] - 2s 35ms/step - loss: 0.8685 - accuracy: 0.7283 - val_loss: 0.9405 - val_accuracy: 0.7140 - lr: 0.0010
Epoch 5/50
55/55 [==============================] - 2s 35ms/step - loss: 0.7107 - accuracy: 0.7766 - val_loss: 0.8598 - val_accuracy: 0.7340 - lr: 0.0010
Epoch 6/50
55/55 [==============================] - 2s 36ms/step - loss: 0.5709 - accuracy: 0.8215 - val_loss: 0.8103 - val_accuracy: 0.7490 - lr: 0.0010
Epoch 7/50
55/55 [==============================] - 2s 36ms/step - loss: 0.4

In [27]:
from keras.models import load_model
model = load_model('best_model_50.h5')
model.evaluate(textcuboid_test,one_hot_test_labels)

118/118 [==============================] - 1s 5ms/step - loss: 0.8410 - accuracy: 0.8062


[0.841046929359436, 0.8061603903770447]